<a href="https://colab.research.google.com/github/ShainaBagri/ClassifyDocumentTopics/blob/main/proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1

In [39]:
!wget 'http://users.csc.calpoly.edu/~foaad/proj1F21_files.zip'
!unzip 'proj1F21_files.zip'

--2021-10-22 21:53:23--  http://users.csc.calpoly.edu/~foaad/proj1F21_files.zip
Resolving users.csc.calpoly.edu (users.csc.calpoly.edu)... 129.65.128.20
Connecting to users.csc.calpoly.edu (users.csc.calpoly.edu)|129.65.128.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129006 (126K) [application/zip]
Saving to: ‘proj1F21_files.zip.2’

proj1F21_files.zip. 100%[===================>] 125.98K   494KB/s    in 0.3s    

2021-10-22 21:53:24 (494 KB/s) - ‘proj1F21_files.zip.2’ saved [129006/129006]

Archive:  proj1F21_files.zip
replace proj1F21_files/proj1F21_1412_A.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: proj1F21_files/proj1F21_1412_A.html  
  inflating: proj1F21_files/proj1F21_1412_B.html  
  inflating: proj1F21_files/proj1F21_1422_A.html  
  inflating: proj1F21_files/proj1F21_1422_B.html  
  inflating: proj1F21_files/proj1F21_1483_A.html  
  inflating: proj1F21_files/proj1F21_1483_B.html  
  inflating: proj1F21_files/proj1F21_1794_A.html  
  

In [40]:
!pip install html2text
!pip install syllables

In [41]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('cmudict')
nltk.download('stopwords')
from nltk.corpus import wordnet as wn
from nltk.metrics.scores import precision, recall, f_measure, accuracy
import pandas as pd
import numpy as np
import os
import random
import re
import html2text
import collections
import syllables
import nltk.corpus
from nltk.corpus import stopwords
from nltk.probability import FreqDist
random.seed(123)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Common Data and Functions

In [42]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
dir = './proj1F21_files/'

In [43]:
def getDocumentData(dir):
  docs = []
  for document in os.listdir(dir):
    dict = {}
    docName = document.split("_")
    dict['author'] = docName[1]
    dict['subject'] = docName[2][0]
    with open(dir + document) as f:
      dict['htmlText'] = f.read()
      dict['rawText'] = html2text.html2text(dict['htmlText'])
      dict['fixedRawText'] = fixRaw(dict['rawText'])
      dict['tokenizedText'] = tokDoc(dict['fixedRawText']) 
    docs.append(dict)
  return docs

def getParagraphData(dir):
  docs = []
  for document in os.listdir(dir):
    docName = document.split("_")
    with open(dir + document) as f:
      htmlText = f.read()
      rawText = html2text.html2text(htmlText)
      text = fixRaw(rawText).split('\n')
      for par in text:
        if par!=[]:
          dict = {}
          dict['author'] = docName[1]
          dict['subject'] = docName[2][0]
          dict['rawText'] = par
          dict['tokenizedText'] = tokPar(dict['rawText']) 
          if dict['tokenizedText']!=[]:
            docs.append(dict)
  return docs

def getSentenceData(dir):
  docs = []
  for document in os.listdir(dir):
    docName = document.split("_")
    with open(dir + document) as f:
      htmlText = f.read()
      rawText = html2text.html2text(htmlText)
      text = fixRaw(rawText).split('\n')
      for par in text:
        sents = sent_tokenizer.tokenize(par)
        for sent in sents:
          dict = {}
          dict['author'] = docName[1]
          dict['subject'] = docName[2][0]
          dict['rawText'] = sent
          dict['tokenizedText'] = tokSent(dict['rawText']) 
          docs.append(dict)
  return docs

# split into training and test sets
# takes in dictionary
# returns pandas dataframes
def splitTrainTest(data):
  random.shuffle(data)
  split = int(len(data) *0.8)
  test_set = pd.DataFrame(data[split:])
  training_set = pd.DataFrame(data[:split])
  return training_set, test_set

In [44]:
def fixRaw(text):
  newText = ""
  for p in re.split('\n\n+', text):
    newText = newText + p.replace('\n', ' ').strip() + '\n'
  return newText

def tokSent(text):
  return [word.lower() for word in nltk.word_tokenize(text)]

def tokPar(text):
  sents = sent_tokenizer.tokenize(text)
  goodSents = []
  for sent in sents:
    goodSents.append(tokSent(sent))
  return goodSents

def tokDoc(text):
  goodParas = []
  for para in text.split('\n'):
    goodParas.append(tokPar(para))
  return goodParas

In [45]:
def getRelatedWords(subject, depth):
    """
    Returns a list of words related to the given subject. Uses wordnet and explores
    hyponyms up to given depth.

    Reccomended depth of 4 for vehicle and 1 for person
    """
    words = set({})
    for c in wn.synsets(subject, pos=wn.NOUN):
        words |= recGetRelated(c, depth)
 
    return words
    

def recGetRelated(sub, depth):
    """
    Recursively gets the related words up to given depth
    """
    if depth == 0:
        return set({})
    
    concepts = [sub]
    concepts += sub.hyponyms()
    
    words = set({})
    for c in concepts:
        words |= recGetRelated(c, depth - 1) # recursive call
        for l in c.lemmas():
            name = l.name()
            if '_' not in name:
                # removes multi word lemmas
                words.add(name)
    
    return words

vehicleCat = getRelatedWords("vehicle", 4)
personCat = getRelatedWords("person", 1) | {"born"}

In [46]:
# for development purposes
def crossValTestSubject(training_set):
  num_folds = 10
  subset = int(len(training_set)/num_folds)
  nbAccuracy = []
  nbPrecision = []
  nbRecall = []
  nbF1 = []
  dtAccuracy = []
  dtPrecision = []
  dtRecall = []
  dtF1 = []
  for i in range(num_folds):
      test_fold = training_set.loc[i*subset:(i+1)*subset]
      train_fold = pd.concat([training_set.loc[:i*subset], training_set.loc[(i+1)*subset:]])
      featureset_train = list(zip(train_fold['features'],train_fold['subject']))
      featureset_test = list(zip(test_fold['features'],test_fold['subject']))
      naiveBayes = nltk.classify.NaiveBayesClassifier.train(featureset_train)
      decisionTree = nltk.classify.DecisionTreeClassifier.train(featureset_train)

      expected = collections.defaultdict(set)
      nbPred = collections.defaultdict(set)
      dtPred = collections.defaultdict(set)

      for j, (feats, label) in enumerate(featureset_test):
        expected[label].add(j)
        observed = naiveBayes.classify(feats)
        nbPred[observed].add(j)
        observed2 = decisionTree.classify(feats)
        dtPred[observed2].add(j)
      
      nbAccuracy.append(nltk.classify.accuracy(naiveBayes, featureset_test))
      dtAccuracy.append(nltk.classify.accuracy(decisionTree, featureset_test))
      for k in expected:
        nbPrecision.append(precision(expected[k], nbPred[k]))
        nbRecall.append(recall(expected[k], nbPred[k]))
        nbF1.append(f_measure(expected[k], nbPred[k]))

        dtPrecision.append(precision(expected[k], dtPred[k]))
        dtRecall.append(recall(expected[k], dtPred[k]))
        dtF1.append(f_measure(expected[k], dtPred[k]))

  nbAccuracy = [x for x in nbAccuracy if x]
  nbPrecision = [x for x in nbPrecision if x]
  nbRecall = [x for x in nbRecall if x]
  nbF1 = [x for x in nbF1 if x]
  dtAccuracy = [x for x in dtAccuracy if x]
  dtPrecision = [x for x in dtPrecision if x]
  dtRecall = [x for x in dtRecall if x]
  dtF1 = [x for x in dtF1 if x]

  print("CROSS VAL METRICS:")
  print("NB Accuracy:", sum(nbAccuracy)/len(nbAccuracy))
  print("NB Precision:", sum(nbPrecision)/len(nbPrecision))
  print("NB Recall:", sum(nbRecall)/len(nbRecall))
  print("NB F1:", sum(nbF1)/len(nbF1))
  print("DT Accuracy:", sum(dtAccuracy)/len(dtAccuracy))
  print("DT Precision:", sum(dtPrecision)/len(dtPrecision))
  print("DT Recall:", sum(dtRecall)/len(dtRecall))
  print("DT F1:", sum(dtF1)/len(dtF1))

In [47]:
# actual metrics
def actualMetricsSubject(training_set, test_set):
    nbPrecision = []
    nbRecall = []
    nbF1 = []
    # dtPrecision = []
    # dtRecall = []
    # dtF1 = []

    featureset_train = list(zip(training_set['features'],training_set['subject']))
    featureset_test = list(zip(test_set['features'],test_set['subject']))
    naiveBayes = nltk.classify.NaiveBayesClassifier.train(featureset_train)
    # decisionTree = nltk.classify.DecisionTreeClassifier.train(featureset_train)

    expected = collections.defaultdict(set)
    nbPred = collections.defaultdict(set)
    # dtPred = collections.defaultdict(set)

    for j, (feats, label) in enumerate(featureset_test):
      expected[label].add(j)
      observed = naiveBayes.classify(feats)
      nbPred[observed].add(j)
      # observed2 = decisionTree.classify(feats)
      # dtPred[observed2].add(j)
      
      nbAccuracy = nltk.classify.accuracy(naiveBayes, featureset_test)
      # dtAccuracy = nltk.classify.accuracy(decisionTree, featureset_test)
      for k in expected:
        nbPrecision.append(precision(expected[k], nbPred[k]))
        nbRecall.append(recall(expected[k], nbPred[k]))
        nbF1.append(f_measure(expected[k], nbPred[k]))

        # dtPrecision.append(precision(expected[k], dtPred[k]))
        # dtRecall.append(recall(expected[k], dtPred[k]))
        # dtF1.append(f_measure(expected[k], dtPred[k]))

    nbPrecision = [x for x in nbPrecision if x]
    nbRecall = [x for x in nbRecall if x]
    nbF1 = [x for x in nbF1 if x]
    # dtPrecision = [x for x in dtPrecision if x]
    # dtRecall = [x for x in dtRecall if x]
    # dtF1 = [x for x in dtF1 if x]

    print("PREDICTIONS:")
    print(nbPred)
    print()
    print("ACTUAL METRICS:")
    print("NB Accuracy:", nbAccuracy)
    print("NB Precision:", sum(nbPrecision)/len(nbPrecision))
    print("NB Recall:", sum(nbRecall)/len(nbRecall))
    print("NB F1:", sum(nbF1)/len(nbF1))
    # print()
    # print("DT Accuracy:", dtAccuracy)
    # print("DT Precision:", sum(dtPrecision)/len(dtPrecision))
    # print("DT Recall:", sum(dtRecall)/len(dtRecall))
    # print("DT F1:", sum(dtF1)/len(dtF1))

In [48]:
# for development purposes
def crossValTestAuthor(training_set):
  num_folds = 10
  subset = int(len(training_set)/num_folds)
  nbAccuracy = []
  nbPrecision = []
  nbRecall = []
  nbF1 = []
  dtAccuracy = []
  dtPrecision = []
  dtRecall = []
  dtF1 = []
  for i in range(num_folds):
      test_fold = training_set.loc[i*subset:(i+1)*subset]
      train_fold = pd.concat([training_set.loc[:i*subset], training_set.loc[(i+1)*subset:]])
      featureset_train = list(zip(train_fold['features'],train_fold['author']))
      featureset_test = list(zip(test_fold['features'],test_fold['author']))
      naiveBayes = nltk.classify.NaiveBayesClassifier.train(featureset_train)
      decisionTree = nltk.classify.DecisionTreeClassifier.train(featureset_train)

      expected = collections.defaultdict(set)
      nbPred = collections.defaultdict(set)
      dtPred = collections.defaultdict(set)

      for j, (feats, label) in enumerate(featureset_test):
        expected[label].add(j)
        observed = naiveBayes.classify(feats)
        nbPred[observed].add(j)
        observed2 = decisionTree.classify(feats)
        dtPred[observed2].add(j)
      
      nbAccuracy.append(nltk.classify.accuracy(naiveBayes, featureset_test))
      dtAccuracy.append(nltk.classify.accuracy(decisionTree, featureset_test))
      for k in expected:
        nbPrecision.append(precision(expected[k], nbPred[k]))
        nbRecall.append(recall(expected[k], nbPred[k]))
        nbF1.append(f_measure(expected[k], nbPred[k]))

        dtPrecision.append(precision(expected[k], dtPred[k]))
        dtRecall.append(recall(expected[k], dtPred[k]))
        dtF1.append(f_measure(expected[k], dtPred[k]))

  nbAccuracy = [x for x in nbAccuracy if x]
  nbPrecision = [x for x in nbPrecision if x]
  nbRecall = [x for x in nbRecall if x]
  nbF1 = [x for x in nbF1 if x]
  dtAccuracy = [x for x in dtAccuracy if x]
  dtPrecision = [x for x in dtPrecision if x]
  dtRecall = [x for x in dtRecall if x]
  dtF1 = [x for x in dtF1 if x]

  print("CROSS VAL METRICS:")
  print("NB Accuracy:", sum(nbAccuracy)/len(nbAccuracy))
  print("NB Precision:", sum(nbPrecision)/len(nbPrecision))
  print("NB Recall:", sum(nbRecall)/len(nbRecall))
  print("NB F1:", sum(nbF1)/len(nbF1))
  print("DT Accuracy:", sum(dtAccuracy)/len(dtAccuracy))
  print("DT Precision:", sum(dtPrecision)/len(dtPrecision))
  print("DT Recall:", sum(dtRecall)/len(dtRecall))
  print("DT F1:", sum(dtF1)/len(dtF1))

In [49]:
# actual metrics
def actualMetricsAuthor(training_set, test_set):
    nbPrecision = []
    nbRecall = []
    nbF1 = []
    # dtPrecision = []
    # dtRecall = []
    # dtF1 = []

    featureset_train = list(zip(training_set['features'],training_set['author']))
    featureset_test = list(zip(test_set['features'],test_set['author']))
    naiveBayes = nltk.classify.NaiveBayesClassifier.train(featureset_train)
    decisionTree = nltk.classify.DecisionTreeClassifier.train(featureset_train)

    expected = collections.defaultdict(set)
    nbPred = collections.defaultdict(set)
    dtPred = collections.defaultdict(set)

    for j, (feats, label) in enumerate(featureset_test):
      expected[label].add(j)
      observed = naiveBayes.classify(feats)
      nbPred[observed].add(j)
      observed2 = decisionTree.classify(feats)
      dtPred[observed2].add(j)
      
      nbAccuracy = nltk.classify.accuracy(naiveBayes, featureset_test)
      dtAccuracy = nltk.classify.accuracy(decisionTree, featureset_test)
      for k in expected:
        nbPrecision.append(precision(expected[k], nbPred[k]))
        nbRecall.append(recall(expected[k], nbPred[k]))
        nbF1.append(f_measure(expected[k], nbPred[k]))

        dtPrecision.append(precision(expected[k], dtPred[k]))
        dtRecall.append(recall(expected[k], dtPred[k]))
        dtF1.append(f_measure(expected[k], dtPred[k]))

    nbPrecision = [x for x in nbPrecision if x]
    nbRecall = [x for x in nbRecall if x]
    nbF1 = [x for x in nbF1 if x]
    dtPrecision = [x for x in dtPrecision if x]
    dtRecall = [x for x in dtRecall if x]
    dtF1 = [x for x in dtF1 if x]

    print("PREDICTIONS:")
    print(nbPred)
    print()
    print("ACTUAL METRICS:")
    print("NB Accuracy:", nbAccuracy)
    print("NB Precision:", sum(nbPrecision)/len(nbPrecision))
    print("NB Recall:", sum(nbRecall)/len(nbRecall))
    print("NB F1:", sum(nbF1)/len(nbF1))
    print()
    print("DT Accuracy:", dtAccuracy)
    print("DT Precision:", sum(dtPrecision)/len(dtPrecision))
    print("DT Recall:", sum(dtRecall)/len(dtRecall))
    print("DT F1:", sum(dtF1)/len(dtF1))

## Identify Subject by Document

In [50]:
def extractFeaturesDoc(tokText, vehicleCat, personCat):
  """
  Extracts features from a document and returns them as a dictionary where
  vehicleCat and personCat are sets of words related to the subjects
  """
 
  features = {}

  # Wordnet words related to person or vehicle
  vehicleCount = sum([sum([len([w for w in s if w in vehicleCat]) for s in p]) for p in tokText])
  personCount = sum([sum([len([w for w in s if w in personCat]) for s in p]) for p in tokText])
  lenDoc = sum([sum([len(s) for s in p]) for p in tokText])

  relatedThresh = 0.005
  features['vehicleRelatedWC'] = 1 if vehicleCount / lenDoc > relatedThresh else 0
  features['personRelatedWC'] = 1 if personCount / lenDoc > relatedThresh else 0

  return features

In [51]:
data = getDocumentData(dir)
training_set, test_set = splitTrainTest(data)
training_set['features'] = training_set.apply(lambda x: extractFeaturesDoc(x['tokenizedText'], vehicleCat, personCat), axis=1)
test_set['features'] = test_set.apply(lambda x: extractFeaturesDoc(x['tokenizedText'], vehicleCat, personCat), axis=1)
crossValTestSubject(training_set)
print()
actualMetricsSubject(training_set, test_set)

CROSS VAL METRICS:
NB Accuracy: 0.9666666666666666
NB Precision: 0.9775
NB Recall: 0.9583333333333334
NB F1: 0.9606349206349207
DT Accuracy: 0.9666666666666666
DT Precision: 0.9775
DT Recall: 0.9583333333333334
DT F1: 0.9606349206349207

PREDICTIONS:
defaultdict(<class 'set'>, {'A': {0, 1, 3, 4, 6, 8, 9, 10, 12}, 'B': {2, 5, 7, 11, 13}})

ACTUAL METRICS:
NB Accuracy: 1.0
NB Precision: 1.0
NB Recall: 1.0
NB F1: 1.0


## Identify Subject by Paragraph

In [52]:
def extractFeaturesPar(tokText, vehicleCat, personCat):
  """
  Extracts features from a document and returns them as a dictionary where
  vehicleCat and personCat are sets of words related to the subjects
  """
 
  features = {}

  # Wordnet words related to person or vehicle
  vehicleCount = sum([len([w for w in s if w in vehicleCat]) for s in tokText])
  personCount = sum([len([w for w in s if w in personCat]) for s in tokText])
  lenDoc = sum([len(s) for s in tokText])

  relatedThresh = 0.005
  features['vehicleRelatedWC'] = 1 if vehicleCount / lenDoc > relatedThresh else 0
  features['personRelatedWC'] = 1 if personCount / lenDoc > relatedThresh else 0

  return features

In [53]:
data = getParagraphData(dir)
training_set, test_set = splitTrainTest(data)
training_set['features'] = training_set.apply(lambda x: extractFeaturesPar(x['tokenizedText'], vehicleCat, personCat), axis=1)
test_set['features'] = test_set.apply(lambda x: extractFeaturesPar(x['tokenizedText'], vehicleCat, personCat), axis=1)
crossValTestSubject(training_set)
print()
actualMetricsSubject(training_set, test_set)

CROSS VAL METRICS:
NB Accuracy: 0.9407407407407407
NB Precision: 0.9470373131770191
NB Recall: 0.9404308885558887
NB F1: 0.9392867656390796
DT Accuracy: 0.9407407407407407
DT Precision: 0.9470373131770191
DT Recall: 0.9404308885558887
DT F1: 0.9392867656390796

PREDICTIONS:
defaultdict(<class 'set'>, {'A': {0, 1, 3, 4, 5, 6, 7, 10, 11, 12, 15, 20, 21, 24, 25, 27, 28, 30, 35, 38, 39, 42, 44, 45, 47, 49, 50, 51, 52, 53, 54, 56, 57, 59, 60, 61, 62, 66, 67}, 'B': {2, 8, 9, 13, 14, 16, 17, 18, 19, 22, 23, 26, 29, 31, 32, 33, 34, 36, 37, 40, 41, 43, 46, 48, 55, 58, 63, 64, 65}})

ACTUAL METRICS:
NB Accuracy: 0.9117647058823529
NB Precision: 0.8733169519145537
NB Recall: 0.864711753648792
NB F1: 0.8655611305147388


## Identify Subject by Sentence

In [54]:
def extractFeaturesSent(tokText, vehicleCat, personCat):
  """
  Extracts features from a document and returns them as a dictionary where
  vehicleCat and personCat are sets of words related to the subjects
  """
 
  features = {}

  # Wordnet words related to person or vehicle
  vehicleCount = len([w for w in tokText if w in vehicleCat])
  personCount = len([w for w in tokText if w in personCat])
  lenDoc = len(tokText)

  features['vehicleRelatedWC'] = 1 if vehicleCount >= 1 else 0
  features['personRelatedWC'] = 1 if personCount >= 1 else 0
  # features['containsNum'] = 1 if len([w for w in tokText if w.isnumeric()]) > 1 else 0

  return features

In [55]:
data = getSentenceData(dir)
training_set, test_set = splitTrainTest(data)
training_set['features'] = training_set.apply(lambda x: extractFeaturesSent(x['tokenizedText'], vehicleCat, personCat), axis=1)
crossValTestSubject(training_set)

CROSS VAL METRICS:
NB Accuracy: 0.7280201342281879
NB Precision: 0.8094615601683948
NB Recall: 0.7278545495580319
NB F1: 0.7067505363577231
DT Accuracy: 0.7280201342281879
DT Precision: 0.8094615601683948
DT Recall: 0.7278545495580319
DT F1: 0.7067505363577231


## Identify Author by Document

In [56]:
stopWords = stopwords.words('english')

def getTopWordsDoc(features, n, doc):
    """
    Gets the n most common words and adds it to the feature list.
    Does not include non alpha words or stopwords
    """

    # Filter stopwords and get frequency
    words = []
    for p in doc:
        for s in p:
            for w in s:
                if w.isalpha() and w not in stopWords:
                    words.append(w)

    freq = [w[0] for w in FreqDist(words).most_common(n)]

    for i in range(n):
        features["topWord" + str(i)] = freq[i]

def extractFeaturesAuthDoc(tokText):
  """
  Extracts features from a document and returns them as a dictionary where
  vehicleCat and personCat are sets of words related to the subjects
  """
 
  features = {}

  avgParLen = sum([len(p) for p in tokText])/len(tokText)
  avgSenLen = sum([sum([len(s) for s in p]) for p in tokText])/sum([len(p) for p in tokText])
  numCommas = sum([sum([len([c for c in s if c==","]) for s in p]) for p in tokText])

  features['avgParLen'] = avgParLen
  features['avgSenLen'] = avgSenLen
  # features['numCommas'] = numCommas//5
  getTopWordsDoc(features, 3, tokText)

  return features

In [61]:
data = getDocumentData(dir)
training_set, test_set = splitTrainTest(data)
training_set['features'] = training_set.apply(lambda x: extractFeaturesAuthDoc(x['tokenizedText']), axis=1)
crossValTestAuthor(training_set)

CROSS VAL METRICS:
NB Accuracy: 0.33333333333333337
NB Precision: 0.875
NB Recall: 0.95
NB F1: 0.8833333333333334
DT Accuracy: 0.35000000000000003
DT Precision: 0.9642857142857143
DT Recall: 0.9523809523809523
DT F1: 0.9396825396825397


## Identify Author by Paragraph

In [69]:
def getTopWordsPara(features, n, doc):
    """
    Gets the n most common words and adds it to the feature list.
    Does not include non alpha words or stopwords
    """

    # Filter stopwords and get frequency
    words = []
    for s in doc:
      for w in s:
        if w.isalpha() and w not in stopWords:
            words.append(w)

    freq = [w[0] for w in FreqDist(words).most_common(n)]

    for i in range(min(n, len(freq))):
        features["topWord" + str(i)] = freq[i]

syllable_dict= nltk.corpus.cmudict.dict() 

def get_syllables(word):
  if word.lower() in syllable_dict.keys():
    syl_count = 0
    syl_list = syllable_dict[word.lower()][0]
    for ele in syl_list:
      if ele[-1].isdigit():
        syl_count += 1
    return syl_count
  else:
    return syllables.estimate(word) 

def isStopword(word):
  if word in stopWords:
    return 1
  else:
    return 0

In [84]:
def extractFeaturesAuthPar(tokText):
  """
  Extracts features from a document and returns them as a dictionary where
  vehicleCat and personCat are sets of words related to the subjects
  """
 
  features = {}

  avgParLen = sum([len(p) for p in tokText])/len(tokText)
  avgSenLen = sum([sum([len(s) for s in p]) for p in tokText])/sum([len(p) for p in tokText])
  avgSyl = sum([sum([sum([get_syllables(w) for w in s]) for s in p]) for p in tokText])/sum([sum([len(s) for s in p]) for p in tokText])
  avgUniqueWords= sum([sum([len(set(s.split())) for s in p]) for p in tokText])/sum([sum([len(s) for s in p]) for p in tokText])
  avgStopWords = sum([sum([sum([isStopword(w) for w in s]) for s in p]) for p in tokText])/sum([sum([len(s) for s in p]) for p in tokText])
  


  features['avgParLen'] = avgParLen
  features['avgSenLen'] = avgSenLen
  features['avgSyl'] = 1 if (avgSyl) <= 3 else 0
  print(avgSyl)
  # features['avgUniqueWords'] = 1 if(avgUniqueWords) >= 5 else 0
  features['avgUniqueWords'] = round(avgUniqueWords * 10)
  # features['avgStopWords'] = 1 if(avgStopWords) <=5 else 0
  features['avgStopWords'] = round(avgStopWords * 10)
  getTopWordsPara(features, 5, tokText)

#ah;ldsfjas;ldjf

  return features

In [86]:
data = getParagraphData(dir)
training_set, test_set = splitTrainTest(data)
training_set['features'] = training_set.apply(lambda x: extractFeaturesAuthPar(x['tokenizedText']), axis=1)
crossValTestAuthor(training_set)

1.0187793427230047
1.0616570327552985
1.0151975683890577
1.0485933503836318
1.0270880361173815
1.0436456996148908
1.0459016393442624
1.037079953650058
1.0163934426229508
1.0485981308411214
1.0372670807453417
1.0330827067669173
1.0185185185185186
1.0727272727272728
1.0401854714064915
1.024390243902439
1.0261865793780687
1.0392156862745099
1.0357142857142858
1.0639834881320949
1.0260756192959584
1.0486725663716814
1.0635593220338984
1.0472972972972974
1.0528455284552845
1.0315581854043392
1.0282186948853616
1.0350194552529184
1.0
1.044776119402985
1.02020202020202
1.0463576158940397
1.0368098159509203
1.0040899795501022
1.0081632653061225
1.0322061191626408
1.0341463414634147
1.0693877551020408
1.0307692307692307
1.0857142857142856
1.051150895140665
1.0344827586206897
1.0387722132471728
1.0392156862745099
1.0443037974683544
1.03690036900369
1.0717948717948718
1.0356687898089172
1.0398406374501992
1.0226628895184136
1.0833333333333333
1.048951048951049
1.0560224089635855
1.048411497730711